In [82]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import SensitivityAtSpecificity
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, matthews_corrcoef
from sklearn.preprocessing import Binarizer
from imblearn.over_sampling import ADASYN, BorderlineSMOTE
from imblearn.combine import SMOTEENN


from sklearn.model_selection import StratifiedKFold


import numpy as np
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    precision_score,
    recall_score,
    matthews_corrcoef,
    confusion_matrix,
    roc_curve
)



def calculate_evaluation_metrics(y_true, y_pred_probs, target_specificity):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_probs)
    specificity = 1 - fpr
    # 找到最接近目标特异性的阈值索引
    closest_index = np.argmin(np.abs(specificity - target_specificity))
    best_threshold = thresholds[closest_index]
    y_pred = y_pred_probs >= best_threshold
    
    acc = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_probs)
    precision = precision_score(y_true, y_pred)
    sn = recall_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    
    return acc, auc, precision, sn, mcc, best_threshold

def calculate(y_true, y_pred_probs):
    y_true = np.array(y_true)
    y_pred_probs = np.array(y_pred_probs)
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_probs)
    specificity = 1 - fpr


    best_threshold = 0.5
    y_pred = y_pred_probs >= best_threshold
    
    acc = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_probs)
    precision = precision_score(y_true, y_pred)
    sn = recall_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    
    return acc, auc, precision, sn, mcc

In [2]:
RACE_Samples = pd.read_csv("TCGA_RACE.csv")

In [19]:
WHITE = RACE_Samples[RACE_Samples["RACE"] == "WHITE"]["A0_Samples"].values
BLACK = RACE_Samples[RACE_Samples["RACE"] == "BLACK"]["A0_Samples"].values
all_sim = set(pd.read_csv(r"最终数据/tcga_clinical.csv", index_col=0).T.columns)
WHITE = list(all_sim & set(WHITE))
BLACK = list(all_sim & set(BLACK))

In [83]:
WHITE = BLACK

In [84]:
TCGA_clinical = pd.read_csv(r"最终数据/tcga_clinical.csv", index_col=0).T[WHITE].T.values
TCGA_clinical = MinMaxScaler().fit_transform(TCGA_clinical)
TCGA_label = pd.read_csv(r"最终数据/tcga_label.csv", index_col=0).T[WHITE].T["os_label"].values
TCGA_gene = pd.read_csv(r"最终数据/final_tcga_gene.csv", index_col=0).T[WHITE].T.values
TCGA_cna = pd.read_csv(r"最终数据/final_tcga_cna.csv", index_col=0).T[WHITE].T.values

TCGA_label = TCGA_label.astype(int)

In [38]:
import pickle
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

for i, (train_index, test_index) in enumerate(skf.split(TCGA_cna, TCGA_label)):
    with open(f'WHITE_train_index_fold{i}.pkl', 'wb') as f:
        pickle.dump(train_index, f)
    with open(f'WHITE_test_index_fold{i}.pkl', 'wb') as f:
        pickle.dump(test_index, f)

# 临床信息

In [43]:
import tensorflow as tf
from tensorflow.keras import Model, initializers
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, GlobalAveragePooling1D, Dense, Dropout, Concatenate, Layer
from tensorflow.keras.initializers import Constant
from tensorflow.keras.activations import sigmoid
from keras.models import Model
from keras.layers import Conv1D, Flatten, Dense, Input
from keras import initializers, regularizers
from keras import layers, models, initializers, backend as K
from keras.activations import softmax
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Concatenate, Dropout, Flatten, Dense
from tensorflow.keras.initializers import GlorotNormal, Constant
from tensorflow.keras.regularizers import l1
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import layers, initializers

class ExternalAttention(layers.Layer):
    def __init__(self, d_model=96, S=64, **kwargs):
        super(ExternalAttention, self).__init__(**kwargs)
        self.d_model = d_model
        self.S = S
        self.mk = layers.Dense(S, use_bias=False, kernel_initializer=initializers.RandomNormal(stddev=0.001))
        self.mv = layers.Dense(self.d_model, use_bias=False, kernel_initializer=initializers.RandomNormal(stddev=0.001))

    def call(self, queries):
        attn = self.mk(queries)  # [bs, n, S]
        attn = tf.nn.softmax(attn)  # [bs, n, S]
        attn = attn / tf.reduce_sum(attn, axis=2, keepdims=True)  # Normalize the softmax output [bs, n, S]
        out = self.mv(attn)  # [bs, n, d_model]
        return out

    def get_config(self):
        config = super(ExternalAttention, self).get_config()
        config.update({
            'd_model': self.d_model,
            'S': self.S
        })
        return config 
    
    
class ChannelGatingLayer(Layer):
    def __init__(self, channels, **kwargs):
        super(ChannelGatingLayer, self).__init__(**kwargs)
        self.channels = channels
        self.gamma = self.add_weight(shape=(1, 1, channels), initializer=Constant(value=1.0), trainable=True)
        self.beta = self.add_weight(shape=(1, 1, channels), initializer=Constant(value=0.0), trainable=True)

    def call(self, inputs, **kwargs):
        # Channel gating computation
        avg = GlobalAveragePooling1D()(inputs)
        avg = tf.expand_dims(avg, axis=1)  # Make it 3D to match with gamma and beta
        avg = tf.add(tf.multiply(avg, self.gamma), self.beta)
        avg = tf.sigmoid(avg)
        
        # Apply gating to the inputs
        return inputs * avg

def create_clincal_model(input_shape):
    init = GlorotNormal(seed=1)
    bias_init = Constant(value=0.1)
    
    inputs = Input(shape=input_shape)

    x = Conv1D(filters=10, kernel_size=2, strides=1, padding="same",
               kernel_initializer=init, bias_initializer=bias_init)(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    residual = x

    xp1 = Conv1D(filters=10, kernel_size=3, strides=1, padding="same", use_bias=False)(x)
    xp1 = BatchNormalization()(xp1)

    xp2 = Conv1D(filters=10, kernel_size=3, strides=1, dilation_rate=2, padding="same", use_bias=False)(x)
    xp2 = BatchNormalization()(xp2)

    x = Concatenate(axis=-1)([xp1, xp2])
    x = Dropout(0.5)(x) 
 
    # Assuming ExternalAttention is a custom layer that you've defined elsewhere
    x = ExternalAttention(d_model=20)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.25)(x)  

    x = x + Conv1D(filters=20, kernel_size=1, strides=1, use_bias=False)(residual)
    x = BatchNormalization()(x)
   
    x = Flatten()(x)

    x = Dense(units=300, activation="tanh", kernel_regularizer=l1(0.01))(x)
    x = Dropout(0.5)(x)  
    outputs = Dense(units=1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["accuracy"])

    return model


In [44]:
from sklearn.utils.class_weight import compute_sample_weight

fold_no = 1
scores = []
all_y_true = []
all_y_prob = []
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
all_fold_histories = []


for i in range(5):  
    with open(f'WHITE_train_index_fold{i}.pkl', 'rb') as f:
        train_index = pickle.load(f)
    with open(f'WHITE_test_index_fold{i}.pkl', 'rb') as f:
        test_index = pickle.load(f)
    
    X_train, X_test = TCGA_clinical[train_index], TCGA_clinical[test_index]
    y_train, y_test = TCGA_label[train_index], TCGA_label[test_index]
    
    smote_enn = ADASYN(random_state=42)

    X_train, y_train = smote_enn.fit_resample(X_train, y_train)
    
    model = create_clincal_model(input_shape = (11, 1))
    print(f'Training on fold {fold_no}...')
    history = model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=50, batch_size=8, verbose=1)
    all_fold_histories.append(history.history)
    # 评估模型
    scores = model.evaluate(X_test, y_test, verbose=0)
    y_pred_prob = model.predict(X_test).reshape(-1)
    all_y_true.extend(y_test)
    all_y_prob.extend(y_pred_prob)
    metrics = calculate_evaluation_metrics(y_test, y_pred_prob, 0.95)
    print('Score fo/r fold',fold_no,metrics)
    fold_no += 1
print(calculate_evaluation_metrics(all_y_true, all_y_prob, 0.95))
pd.DataFrame([all_y_true,all_y_prob]).T.to_csv("WHITE_clincal.csv")
# wu(0.8811513463324049, 0.8787080689909397, 0.7980295566502463, 0.6506024096385542, 0.6479587505455906, 0.53422964)
# (0.8644382544103992, 0.887356188037173, 0.7783783783783784, 0.5783132530120482, 0.5910646621570037, 0.89936125)


(0.8679245283018868, 0.8932682963295209, 0.810126582278481, 0.6530612244897959, 0.644132086346931, 0.7824894)

Training on fold 1...
Epoch 1/50
108/108 [==============================] - 1s 6ms/step - loss: 17.9844 - accuracy: 0.7428 - val_loss: 5.8938 - val_accuracy: 0.3557
Epoch 2/50
108/108 [==============================] - 0s 3ms/step - loss: 2.7083 - accuracy: 0.7845 - val_loss: 1.6281 - val_accuracy: 0.3423
Epoch 3/50
108/108 [==============================] - 0s 3ms/step - loss: 1.0907 - accuracy: 0.7949 - val_loss: 1.1260 - val_accuracy: 0.6376
Epoch 4/50
108/108 [==============================] - 0s 3ms/step - loss: 0.8742 - accuracy: 0.8100 - val_loss: 1.0172 - val_accuracy: 0.6376
Epoch 5/50
108/108 [==============================] - 0s 3ms/step - loss: 0.8988 - accuracy: 0.7961 - val_loss: 0.9759 - val_accuracy: 0.7919
Epoch 6/50
108/108 [==============================] - 0s 3ms/step - loss: 0.9341 - accuracy: 0.7891 - val_loss: 0.7907 - val_accuracy: 0.8591
Epoch 7/50
108/108 [==============================] - 0s 3ms/step - loss: 0.8432 - accuracy: 0.8019 - val_loss: 0.8408 - val_

110/110 [==============================] - 0s 2ms/step - loss: 0.7337 - accuracy: 0.8295 - val_loss: 0.8075 - val_accuracy: 0.7973
Epoch 14/50
110/110 [==============================] - 0s 2ms/step - loss: 0.7425 - accuracy: 0.8159 - val_loss: 0.7216 - val_accuracy: 0.8446
Epoch 15/50
110/110 [==============================] - 0s 2ms/step - loss: 0.7825 - accuracy: 0.8000 - val_loss: 0.6882 - val_accuracy: 0.8311
Epoch 16/50
110/110 [==============================] - 0s 2ms/step - loss: 0.7096 - accuracy: 0.8261 - val_loss: 0.6789 - val_accuracy: 0.8514
Epoch 17/50
110/110 [==============================] - 0s 2ms/step - loss: 0.7295 - accuracy: 0.8216 - val_loss: 0.7351 - val_accuracy: 0.7973
Epoch 18/50
110/110 [==============================] - 0s 2ms/step - loss: 0.7039 - accuracy: 0.8352 - val_loss: 0.7008 - val_accuracy: 0.8378
Epoch 19/50
110/110 [==============================] - 0s 3ms/step - loss: 0.7164 - accuracy: 0.8136 - val_loss: 0.6973 - val_accuracy: 0.8243
Epoch 20/50

109/109 [==============================] - 0s 3ms/step - loss: 0.7270 - accuracy: 0.8030 - val_loss: 0.6608 - val_accuracy: 0.8581
Epoch 26/50
109/109 [==============================] - 0s 3ms/step - loss: 0.6971 - accuracy: 0.8191 - val_loss: 0.6832 - val_accuracy: 0.8514
Epoch 27/50
109/109 [==============================] - 0s 3ms/step - loss: 0.7245 - accuracy: 0.8018 - val_loss: 0.6920 - val_accuracy: 0.8378
Epoch 28/50
109/109 [==============================] - 0s 3ms/step - loss: 0.7045 - accuracy: 0.8076 - val_loss: 0.6750 - val_accuracy: 0.8108
Epoch 29/50
109/109 [==============================] - 0s 3ms/step - loss: 0.6976 - accuracy: 0.8191 - val_loss: 0.6629 - val_accuracy: 0.8446
Epoch 30/50
109/109 [==============================] - 0s 3ms/step - loss: 0.6979 - accuracy: 0.8145 - val_loss: 0.8212 - val_accuracy: 0.7703
Epoch 31/50
109/109 [==============================] - 0s 3ms/step - loss: 0.6948 - accuracy: 0.8053 - val_loss: 0.5922 - val_accuracy: 0.8581
Epoch 32/50

# 拷贝数变异

In [60]:
def create_cna_model(input_shape):
    init = GlorotNormal(seed=1)
    bias_init = Constant(value=0.1)
    
    inputs = Input(shape=input_shape)

    x = Conv1D(filters=10, kernel_size=2, strides=1, padding="same",
               kernel_initializer=init, bias_initializer=bias_init)(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    residual = x

    xp1 = Conv1D(filters=10, kernel_size=3, strides=1, padding="same", use_bias=False)(x)
    xp1 = BatchNormalization()(xp1)

    xp2 = Conv1D(filters=10, kernel_size=3, strides=1, dilation_rate=2, padding="same", use_bias=False)(x)
    xp2 = BatchNormalization()(xp2)

    x = Concatenate(axis=-1)([xp1, xp2])
    x = Dropout(0.5)(x) 
 
    # Assuming ExternalAttention is a custom layer that you've defined elsewhere
    x = ExternalAttention(d_model=20)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.25)(x)  

    x = x + Conv1D(filters=20, kernel_size=1, strides=1, use_bias=False)(residual)
    x = BatchNormalization()(x)
   
    x = Flatten()(x)

    x = Dense(units=300, activation="tanh", kernel_regularizer=l1(0.01))(x)
    x = Dropout(0.5)(x)  
    outputs = Dense(units=1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["accuracy"])

    return model


In [72]:
from sklearn.utils.class_weight import compute_sample_weight

fold_no = 1
scores = []
all_y_true = []
all_y_prob = []
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
all_fold_histories = []

for i in range(5):  
    with open(f'WHITE_train_index_fold{i}.pkl', 'rb') as f:
        train_index = pickle.load(f)
    with open(f'WHITE_test_index_fold{i}.pkl', 'rb') as f:
        test_index = pickle.load(f)
        
    X_train, X_test = TCGA_cna[train_index], TCGA_cna[test_index]
    y_train, y_test = TCGA_label[train_index], TCGA_label[test_index]
    adasyn = ADASYN(random_state=42)
    X_train, y_train = adasyn.fit_resample(X_train, y_train)
    
    model = create_cna_model(input_shape = (280, 1))
    print(f'Training on fold {fold_no}...')
    history = model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=50, batch_size=32, verbose=0)
    
    # 评估模型
    all_fold_histories.append(history.history)
    scores = model.evaluate(X_test, y_test, verbose=0)
    y_pred_prob = model.predict(X_test).reshape(-1)
    all_y_true.extend(y_test)
    all_y_prob.extend(y_pred_prob)
    metrics = calculate_evaluation_metrics(y_test, y_pred_prob, 0.95)
    print('Score fo/r fold',fold_no,metrics)
    fold_no += 1
print(calculate_evaluation_metrics(all_y_true, all_y_prob, 0.95))
pd.DataFrame([all_y_true,all_y_prob]).T.to_csv("WHITE_cna.csv")  #H


Training on fold 1...
Score fo/r fold 1 (0.8322147651006712, 0.8778554778554779, 0.7916666666666666, 0.48717948717948717, 0.5282262029878123, 0.6254375)
Training on fold 2...
Score fo/r fold 2 (0.8389261744966443, 0.8802752293577981, 0.8076923076923077, 0.525, 0.5594431565230081, 0.6600033)
Training on fold 3...
Score fo/r fold 3 (0.8378378378378378, 0.9096683133380381, 0.8, 0.5128205128205128, 0.5490253513708236, 0.44331643)
Training on fold 4...
Score fo/r fold 4 (0.8445945945945946, 0.9117854622441778, 0.7857142857142857, 0.5641025641025641, 0.5725877668903961, 0.5911551)
Training on fold 5...
Score fo/r fold 5 (0.8851351351351351, 0.9498941425546931, 0.84375, 0.6923076923076923, 0.6917787099673642, 0.5547613)
(0.8571428571428571, 0.9017997308813634, 0.8125, 0.5969387755102041, 0.6103336331679228, 0.56264615)


# 基因表达

In [80]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Concatenate, Dropout, Flatten, Dense
from tensorflow.keras.initializers import GlorotNormal, Constant
from tensorflow.keras.regularizers import l1
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def create_gene_model(input_shape):
    init = GlorotNormal(seed=1)
    bias_init = Constant(value=0.1)
    
    inputs = Input(shape=input_shape)

    x = Conv1D(filters=25, kernel_size=2, strides=1, padding="same",
               kernel_initializer=init, bias_initializer=bias_init)(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    residual = x

    xp1 = Conv1D(filters=10, kernel_size=3, strides=1, padding="same", use_bias=False)(x)
    xp1 = BatchNormalization()(xp1)

    xp2 = Conv1D(filters=10, kernel_size=3, strides=1, dilation_rate=2, padding="same", use_bias=False)(x)
    xp2 = BatchNormalization()(xp2)

    x = Concatenate(axis=-1)([xp1, xp2])
    x = Dropout(0.5)(x) 
 
    # Assuming ExternalAttention is a custom layer that you've defined elsewhere
#     x = ExternalAttention(d_model=20)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.25)(x)  

    x = x + Conv1D(filters=20, kernel_size=1, strides=1, use_bias=False)(residual)
    x = BatchNormalization()(x)
   
    x = Flatten()(x)

    x = Dense(units=300, activation="tanh", kernel_regularizer=l1(0.01))(x)
    x = Dropout(0.5)(x)  
    outputs = Dense(units=1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    
    optimizer = Adam(learning_rate=0.0003)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["accuracy"])

    return model

In [81]:
from sklearn.utils.class_weight import compute_sample_weight

fold_no = 1
scores = []
all_y_true = []
all_y_prob = []
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
all_fold_histories = []
for i in range(5):  
    with open(f'WHITE_train_index_fold{i}.pkl', 'rb') as f:
        train_index = pickle.load(f)
    with open(f'WHITE_test_index_fold{i}.pkl', 'rb') as f:
        test_index = pickle.load(f)
    
    X_train, X_test = TCGA_gene[train_index], TCGA_gene[test_index]
    y_train, y_test = TCGA_label[train_index], TCGA_label[test_index]
    
    smote_enn = ADASYN(random_state=42)

    X_train, y_train = smote_enn.fit_resample(X_train, y_train)
    
    model = create_gene_model(input_shape = (280, 1))
    print(f'Training on fold {fold_no}...')
    history = model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=50, batch_size=32, verbose=1)
    
    # 评估模型
    all_fold_histories.append(history.history)
    scores = model.evaluate(X_test, y_test, verbose=0)
    y_pred_prob = model.predict(X_test).reshape(-1)
    all_y_true.extend(y_test)
    all_y_prob.extend(y_pred_prob)
    metrics = calculate_evaluation_metrics(y_test, y_pred_prob, 0.95)
    print('Score fo/r fold',fold_no,metrics)
    fold_no += 1
print(calculate_evaluation_metrics(all_y_true, all_y_prob, 0.95))
pd.DataFrame([all_y_true,all_y_prob]).T.to_csv("WHITE_gene.csv")

Training on fold 1...
Epoch 1/50
27/27 [==============================] - 2s 65ms/step - loss: 216.3647 - accuracy: 0.6021 - val_loss: 163.3075 - val_accuracy: 0.7584
Epoch 2/50
27/27 [==============================] - 1s 55ms/step - loss: 124.1912 - accuracy: 0.7816 - val_loss: 85.1801 - val_accuracy: 0.7919
Epoch 3/50
27/27 [==============================] - 2s 56ms/step - loss: 58.2277 - accuracy: 0.8831 - val_loss: 33.4558 - val_accuracy: 0.5302
Epoch 4/50
27/27 [==============================] - 1s 54ms/step - loss: 19.3059 - accuracy: 0.8996 - val_loss: 9.1152 - val_accuracy: 0.5168
Epoch 5/50
27/27 [==============================] - 1s 54ms/step - loss: 5.3409 - accuracy: 0.8949 - val_loss: 3.4183 - val_accuracy: 0.5906
Epoch 6/50
27/27 [==============================] - 2s 57ms/step - loss: 2.8095 - accuracy: 0.8217 - val_loss: 2.8087 - val_accuracy: 0.3758
Epoch 7/50
27/27 [==============================] - 1s 54ms/step - loss: 2.5407 - accuracy: 0.8040 - val_loss: 2.6379 - va

Epoch 14/50
27/27 [==============================] - 2s 64ms/step - loss: 2.9184 - accuracy: 0.7500 - val_loss: 2.8513 - val_accuracy: 0.6081
Epoch 15/50
27/27 [==============================] - 2s 69ms/step - loss: 2.6252 - accuracy: 0.7931 - val_loss: 2.7400 - val_accuracy: 0.6014
Epoch 16/50
27/27 [==============================] - 2s 58ms/step - loss: 2.7886 - accuracy: 0.7811 - val_loss: 2.6797 - val_accuracy: 0.7365
Epoch 17/50
27/27 [==============================] - 2s 68ms/step - loss: 2.6115 - accuracy: 0.8098 - val_loss: 2.6111 - val_accuracy: 0.6622
Epoch 18/50
27/27 [==============================] - 2s 63ms/step - loss: 2.5262 - accuracy: 0.8194 - val_loss: 2.6385 - val_accuracy: 0.6689
Epoch 19/50
27/27 [==============================] - 2s 65ms/step - loss: 2.7193 - accuracy: 0.7907 - val_loss: 2.7684 - val_accuracy: 0.7027
Epoch 20/50
27/27 [==============================] - 2s 57ms/step - loss: 2.7012 - accuracy: 0.8098 - val_loss: 2.9124 - val_accuracy: 0.7297
Epoch 

27/27 [==============================] - 2s 64ms/step - loss: 2.5972 - accuracy: 0.8195 - val_loss: 2.6182 - val_accuracy: 0.7973
Epoch 27/50
27/27 [==============================] - 2s 68ms/step - loss: 2.5039 - accuracy: 0.8219 - val_loss: 2.5429 - val_accuracy: 0.8378
Epoch 28/50
27/27 [==============================] - 2s 69ms/step - loss: 2.5660 - accuracy: 0.8183 - val_loss: 2.5836 - val_accuracy: 0.7365
Epoch 29/50
27/27 [==============================] - 2s 67ms/step - loss: 2.5818 - accuracy: 0.8278 - val_loss: 2.5683 - val_accuracy: 0.7500
Epoch 30/50
27/27 [==============================] - 2s 71ms/step - loss: 2.6119 - accuracy: 0.8219 - val_loss: 2.6821 - val_accuracy: 0.7635
Epoch 31/50
27/27 [==============================] - 2s 65ms/step - loss: 2.5770 - accuracy: 0.8361 - val_loss: 2.5610 - val_accuracy: 0.8108
Epoch 32/50
27/27 [==============================] - 2s 67ms/step - loss: 2.5812 - accuracy: 0.8266 - val_loss: 2.6252 - val_accuracy: 0.7500
Epoch 33/50
27/27 